#Capstone Project -Using Large Language Models to Automate Patent Citation Analysis
---

##CAPSTONE PROJECT REPORT SUBMITTED TO THE
##INDIAN SCHOOL OF BUSINESS
###ADVANCED MANAGEMENT PROGRAMME IN BUSINESS ANALYTICS
---
SUBMITTED BY:
---
1 Archit Goel 12220026 Archit_Goel_ampba2023W@isb.edu
---
2 Arpit Agarwal 12220065 arpit_agarwal_ampba2023w@isb.edu
---
3 Dhruv Puri 12220058 Dhruv_Puri_ampba2023W@isb.edu
---
4 Kunwar Ji Gupta 12220025 kunwar_gupta_ampba2023w@isb.edu
---
5 Paritosh Sinha 12220056 Paritosh_sinha_ampba2023w@isb.edu
---


### UNDER THE GUIDANCE OF
---

#Mr. Bharani Kumar
INSTITUTE OF DATA SCIENCE
INDIAN SCHOOL OF BUSINESS


## Installing Necessary Packages

We are installing and upgrading several essential packages needed for our project with the following commands:


In [ ]:
!pip install --upgrade dask distributed pandas
!pip install pyspellchecker
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install contractions
!pip install datasets
!pip install num2words
!pip install nltk
!pip -q install accelerate
!pip install llama-cpp-python
!pip install scikit-learn
!pip install transformers datasets torch
!pip install bert_score
!pip install rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 64.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
  Attempting uninstall: dask
    Found existing installation: dask 2023.8.1
    Uninstalling dask-2023.8.1:
      Successfully uninstalled dask-2023.8.1
  Attempting uninstall: distributed
    Found existing installation: distributed 2023.8.1
    Uninstalling distributed-2023.8.1:
      Successfully uninstalled distributed-2023.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
go

## Importing Libraries and Downloading Resources

We are importing the necessary libraries and downloading NLTK resources if they are not already downloaded:


In [ ]:
import warnings
import pandas as pd
import re
import nltk
import contractions
import dask.dataframe as dd
from transformers import BertTokenizer, BertModel
import torch
import faiss
import numpy as np
from google.colab import files
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
# Downloading NLTK resources if not already downloaded
nltk.download('punkt')
from nltk.corpus import stopwords
from num2words import num2words
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments, RagTokenForGeneration, RagTokenizer, RagRetriever, RagSequenceForGeneration
from datasets import Dataset, DatasetDict
import gdown
import logging
import torch
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
import logging

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Data Preparation

During the pre-processing stage, the following steps were undertaken:

- **Text-Based Data Cleaning:** Utilizing Natural Language Processing (NLP) libraries for text normalization, spell checking, and punctuation removal.

- **EDA and Feature Engineering:** Key features essential for subsequent model tasks were extracted through exploratory analysis. This step elevated the data value by transforming raw text into an informative feature set, crucial for accurate Large Language Model (LLM) tasks in patent citation analysis.

    - *Quality Verification:* Thorough validation checks were executed at each stage to uphold data integrity and consistency.

Before proceeding with the text cleaning process, it's important to note that the following code blocks are designed for text cleaning. As we are working on a small dataset, we have extracted the complete data in CSV form and are currently operating on a sample set from the actual population.

##Uploading Data

We are importing the necessary libraries and downloading the CSV file from Google Drive. Then, we are loading the CSV file into a DataFrame:


In [ ]:
# Import libraries and upload data
import pandas as pd
import gdown
from google.colab import files

# Download the CSV file from Google Drive
file_id = "1lwT6TtdMXqhNThbO94YosIMqpLtNZCvS"
url = f"https://drive.google.com/uc?id={file_id}"
output = "data.csv"  # Choose the desired output file name

gdown.download(url, output, quiet=False)

# Load the CSV file into a DataFrame
concatenated_df = pd.read_csv(output)
concatenated_df

Downloading...
From: https://drive.google.com/uc?id=1lwT6TtdMXqhNThbO94YosIMqpLtNZCvS
To: /content/data.csv
100%|██████████| 28.5M/28.5M [00:00<00:00, 180MB/s]


,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555473,Dual bladder fuel tank,2023-01-17,"PATENT_TITLE: Dual bladder fuel tank, PATENT_A..."
1,11555474,Pipe section of a common rail line and methods...,2023-01-17,PATENT_TITLE: Pipe section of a common rail li...
2,11555475,Turbine,2023-01-17,"PATENT_TITLE: Turbine, PATENT_ABSTRACT: A turb..."
3,11555476,Wind turbine blade root attachment system and ...,2023-01-17,PATENT_TITLE: Wind turbine blade root attachme...
4,11555477,Bird or bat detection and identification for w...,2023-01-17,PATENT_TITLE: Bird or bat detection and identi...
...,...,...,...,...
995,11560418,Methods and compositions for modified factor I...,2023-01-24,PATENT_TITLE: Methods and compositions for mod...
996,11560438,Porous polymeric cellulose prepared via cellul...,2023-01-24,PATENT_TITLE: Porous polymeric cellulose prepa...
997,11560443,"Grafted polyvinyl alcohol polymer, formulation...",2023-01-24,PATENT_TITLE: Grafted polyvinyl alcohol polyme...
998,11560446,Polyurethane foam article and method of formin...,2023-01-24,PATENT_TITLE: Polyurethane foam article and me...


## Creating and Checking DataFrames

We are creating a training DataFrame with the first 20 rows and another DataFrame with rows 21-30. We are also optionally resetting the index and assigning names to these DataFrames:

In [ ]:
# Create data_train_df with the first 20 rows
data_train_df = concatenated_df.head(20)

# Create df_summ with rows 21-30
df_summ = concatenated_df.iloc[20:30]

# Optionally, if you want to reset the index for both dataframes
data_train_df.reset_index(drop=True, inplace=True)
df_summ.reset_index(drop=True, inplace=True)

# Optionally, if you want to explicitly assign names to the dataframes
data_train_df.name = 'data_train_df'
df_summ.name = 'df_summ'

# Optionally, if you want to check the dataframes
print("data_train_df:")
data_train_df

data_train_df:


,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555473,Dual bladder fuel tank,2023-01-17,"PATENT_TITLE: Dual bladder fuel tank, PATENT_A..."
1,11555474,Pipe section of a common rail line and methods...,2023-01-17,PATENT_TITLE: Pipe section of a common rail li...
2,11555475,Turbine,2023-01-17,"PATENT_TITLE: Turbine, PATENT_ABSTRACT: A turb..."
3,11555476,Wind turbine blade root attachment system and ...,2023-01-17,PATENT_TITLE: Wind turbine blade root attachme...
4,11555477,Bird or bat detection and identification for w...,2023-01-17,PATENT_TITLE: Bird or bat detection and identi...
5,11555478,Deep-sea multi-energy integrated platform for ...,2023-01-17,PATENT_TITLE: Deep-sea multi-energy integrated...
6,11555479,Portable wind energy conversion system and rel...,2023-01-17,PATENT_TITLE: Portable wind energy conversion ...
7,11555480,Prestressed-bolted dry-assembled segmental pre...,2023-01-17,PATENT_TITLE: Prestressed-bolted dry-assembled...
8,11555481,Inspection device controller for an inspection...,2023-01-17,PATENT_TITLE: Inspection device controller for...
9,11555482,Stepped conductivity interface,2023-01-17,"PATENT_TITLE: Stepped conductivity interface, ..."


In [ ]:
print("\ndf_summ:")
df_summ


df_summ:


,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555494,Oldham coupling in co-rotating scroll compressors,2023-01-17,PATENT_TITLE: Oldham coupling in co-rotating s...
1,11555495,Fluid equalisation for multiple compressors,2023-01-17,PATENT_TITLE: Fluid equalisation for multiple ...
2,11555496,Centrifugal pump,2023-01-17,"PATENT_TITLE: Centrifugal pump, PATENT_ABSTRAC..."
3,11555497,Quick startup device for centrifugal pump,2023-01-17,PATENT_TITLE: Quick startup device for centrif...
4,11555498,"Magnetic coupling assemblies and pump, turbine...",2023-01-17,PATENT_TITLE: Magnetic coupling assemblies and...
5,11555499,Centrifugal pump assembly,2023-01-17,"PATENT_TITLE: Centrifugal pump assembly, PATEN..."
6,11555500,Guide vane,2023-01-17,"PATENT_TITLE: Guide vane, PATENT_ABSTRACT: The..."
7,11555501,Ceiling fan structure,2023-01-17,"PATENT_TITLE: Ceiling fan structure, PATENT_AB..."
8,11555502,Compressor including inlet guide vanes,2023-01-17,PATENT_TITLE: Compressor including inlet guide...
9,11555503,Axial counterbalance for rotating components,2023-01-17,PATENT_TITLE: Axial counterbalance for rotatin...


## Extracting Data from Selected Columns

We are extracting data from three columns and creating a new DataFrame with these selected columns for train datasets:

In [ ]:
# Extracting data from three columns
selected_columns = ['PATENT_ID', 'PATENT_TITLE','PATENT_DATE', 'Concatenated_Text']
extracted_data_train = data_train_df[selected_columns]
extracted_data_train

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555473,Dual bladder fuel tank,2023-01-17,"PATENT_TITLE: Dual bladder fuel tank, PATENT_A..."
1,11555474,Pipe section of a common rail line and methods...,2023-01-17,PATENT_TITLE: Pipe section of a common rail li...
2,11555475,Turbine,2023-01-17,"PATENT_TITLE: Turbine, PATENT_ABSTRACT: A turb..."
3,11555476,Wind turbine blade root attachment system and ...,2023-01-17,PATENT_TITLE: Wind turbine blade root attachme...
4,11555477,Bird or bat detection and identification for w...,2023-01-17,PATENT_TITLE: Bird or bat detection and identi...
5,11555478,Deep-sea multi-energy integrated platform for ...,2023-01-17,PATENT_TITLE: Deep-sea multi-energy integrated...
6,11555479,Portable wind energy conversion system and rel...,2023-01-17,PATENT_TITLE: Portable wind energy conversion ...
7,11555480,Prestressed-bolted dry-assembled segmental pre...,2023-01-17,PATENT_TITLE: Prestressed-bolted dry-assembled...
8,11555481,Inspection device controller for an inspection...,2023-01-17,PATENT_TITLE: Inspection device controller for...
9,11555482,Stepped conductivity interface,2023-01-17,"PATENT_TITLE: Stepped conductivity interface, ..."


## Extracting Data from Selected Columns

We are extracting data from three columns and creating a new DataFrame with these selected columns from the summary DataFrame for the test dataset:

In [ ]:
# Extracting data from three columns
selected_columns = ['PATENT_ID', 'PATENT_TITLE','PATENT_DATE', 'Concatenated_Text']
extracted_data_summ = df_summ[selected_columns]
extracted_data_summ

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555494,Oldham coupling in co-rotating scroll compressors,2023-01-17,PATENT_TITLE: Oldham coupling in co-rotating s...
1,11555495,Fluid equalisation for multiple compressors,2023-01-17,PATENT_TITLE: Fluid equalisation for multiple ...
2,11555496,Centrifugal pump,2023-01-17,"PATENT_TITLE: Centrifugal pump, PATENT_ABSTRAC..."
3,11555497,Quick startup device for centrifugal pump,2023-01-17,PATENT_TITLE: Quick startup device for centrif...
4,11555498,"Magnetic coupling assemblies and pump, turbine...",2023-01-17,PATENT_TITLE: Magnetic coupling assemblies and...
5,11555499,Centrifugal pump assembly,2023-01-17,"PATENT_TITLE: Centrifugal pump assembly, PATEN..."
6,11555500,Guide vane,2023-01-17,"PATENT_TITLE: Guide vane, PATENT_ABSTRACT: The..."
7,11555501,Ceiling fan structure,2023-01-17,"PATENT_TITLE: Ceiling fan structure, PATENT_AB..."
8,11555502,Compressor including inlet guide vanes,2023-01-17,PATENT_TITLE: Compressor including inlet guide...
9,11555503,Axial counterbalance for rotating components,2023-01-17,PATENT_TITLE: Axial counterbalance for rotatin...


## Converting All Values to Lowercase

We are converting all string values to lowercase in both the summary and training DataFrames:

In [ ]:
# Converting all values to lowercase
extracted_data_summ = extracted_data_summ.applymap(lambda x: x.lower() if isinstance(x, str) else x)
extracted_data_summ

# Converting all values to lowercase
extracted_data_train = extracted_data_train.applymap(lambda x: x.lower() if isinstance(x, str) else x)
extracted_data_train

<ipython-input-8-435ff100d985>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  extracted_data_summ = extracted_data_summ.applymap(lambda x: x.lower() if isinstance(x, str) else x)
<ipython-input-8-435ff100d985>:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  extracted_data_train = extracted_data_train.applymap(lambda x: x.lower() if isinstance(x, str) else x)


,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555473,dual bladder fuel tank,2023-01-17,"patent_title: dual bladder fuel tank, patent_a..."
1,11555474,pipe section of a common rail line and methods...,2023-01-17,patent_title: pipe section of a common rail li...
2,11555475,turbine,2023-01-17,"patent_title: turbine, patent_abstract: a turb..."
3,11555476,wind turbine blade root attachment system and ...,2023-01-17,patent_title: wind turbine blade root attachme...
4,11555477,bird or bat detection and identification for w...,2023-01-17,patent_title: bird or bat detection and identi...
5,11555478,deep-sea multi-energy integrated platform for ...,2023-01-17,patent_title: deep-sea multi-energy integrated...
6,11555479,portable wind energy conversion system and rel...,2023-01-17,patent_title: portable wind energy conversion ...
7,11555480,prestressed-bolted dry-assembled segmental pre...,2023-01-17,patent_title: prestressed-bolted dry-assembled...
8,11555481,inspection device controller for an inspection...,2023-01-17,patent_title: inspection device controller for...
9,11555482,stepped conductivity interface,2023-01-17,"patent_title: stepped conductivity interface, ..."


## Defining a Function to Fix Contractions

We are defining a function to fix contractions in a text and print the changes:



In [ ]:
# Defining a function to fix contractions in a text and print the changes
def fix_contractions(text):
    original_text = text
    updated_text = contractions.fix(text)
    print("Original text:", original_text)
    print("Updated text:", updated_text)
    return updated_text

# Applying the function to the 'PATENT_TITLE' & 'PATENT_ABSTRACT' column
Currently applying the function to the 'PATENT_TITLE' and PATENT_ABSTRACT column on train dataset.

In [ ]:
# Applying the function to the 'PATENT_TITLE' column
extracted_data_train['PATENT_TITLE'] = extracted_data_train['PATENT_TITLE'].apply(fix_contractions)
# Applying the function to the 'PATENT_ABSTRACT' column
extracted_data_train['Concatenated_Text'] = extracted_data_train['Concatenated_Text'].apply(fix_contractions)

Original text: dual bladder fuel tank
Updated text: dual bladder fuel tank
Original text: pipe section of a common rail line and methods for manufacturing same
Updated text: pipe section of a common rail line and methods for manufacturing same
Original text: turbine
Updated text: turbine
Original text: wind turbine blade root attachment system and method of manufacture
Updated text: wind turbine blade root attachment system and method of manufacture
Original text: bird or bat detection and identification for wind turbine risk mitigation
Updated text: bird or bat detection and identification for wind turbine risk mitigation
Original text: deep-sea multi-energy integrated platform for complementary power generation, production, living and exploration
Updated text: deep-sea multi-energy integrated platform for complementary power generation, production, living and exploration
Original text: portable wind energy conversion system and related techniques
Updated text: portable wind energy co

# Applying the function to the 'PATENT_TITLE' & 'PATENT_ABSTRACT' column
Currently applying the function to the 'PATENT_TITLE' and PATENT_ABSTRACT column on summary dataset.

In [ ]:
# Applying the function to the 'PATENT_TITLE' column
extracted_data_summ['PATENT_TITLE'] = extracted_data_summ['PATENT_TITLE'].apply(fix_contractions)
# Applying the function to the 'PATENT_ABSTRACT' column
extracted_data_summ['Concatenated_Text'] = extracted_data_summ['Concatenated_Text'].apply(fix_contractions)

Original text: oldham coupling in co-rotating scroll compressors
Updated text: oldham coupling in co-rotating scroll compressors
Original text: fluid equalisation for multiple compressors
Updated text: fluid equalisation for multiple compressors
Original text: centrifugal pump
Updated text: centrifugal pump
Original text: quick startup device for centrifugal pump
Updated text: quick startup device for centrifugal pump
Original text: magnetic coupling assemblies and pump, turbine, and compressor including the magnetic coupling assembly
Updated text: magnetic coupling assemblies and pump, turbine, and compressor including the magnetic coupling assembly
Original text: centrifugal pump assembly
Updated text: centrifugal pump assembly
Original text: guide vane
Updated text: guide vane
Original text: ceiling fan structure
Updated text: ceiling fan structure
Original text: compressor including inlet guide vanes
Updated text: compressor including inlet guide vanes
Original text: axial counterb

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

## Text-Based Data Cleaning

The following sections outline the steps involved in text-based data cleaning, ensuring the text data is refined and prepared for advanced analysis:

1. **Spell Checking**
   - *Library Utilization:* The spellchecker library was employed to rectify spelling errors within the text.
   - *Function Implementation:* A dedicated function, `correct_spelling`, was developed to perform spell checking and correction for individual words. This functionality was further extended to correct entire reviews via the `correct_review` function.
   - *Batch Processing:* To efficiently handle large volumes of text, batch processing techniques were implemented.

2. **Text Preprocessing**
   - *Spell Checking and Correction:* Functions were created to manage spell checking and correction for both individual words and complete reviews.
   - *Batch Processing Enhancement:* The implementation of batch processing significantly improved the efficiency of text preprocessing operations.

3. **Specific Phrase Removal**
   - *Identification and Removal:* The recurring phrase "detailed description of the invention" was identified, with an intention to remove or replace this specific phrase throughout the text.

4. **Number Removal**
   - *Numerical Instances:* Specific numerical references such as "conditioner 120" were identified, with an intention to eliminate these instances from the text.

5. **Punctuation Removal**
   - *Stripping Punctuation:* Punctuation marks including commas, periods, exclamation marks, and others were removed. This step is crucial to ensure punctuation does not interfere with subsequent text analysis tasks.

6. **Handling Contractions**
   - *Expansion of Contractions:* Contractions such as "can't" were expanded to "cannot", and "don't" to "do not", enhancing consistency and clarity in the text data.

7. **Removing Special Characters**
   - *Elimination of Non-Alphanumeric Characters:* Special characters, symbols, and other non-alphanumeric characters were removed from the text. These characters often do not contribute meaningfully to text analysis tasks and thus were excluded.

These steps collectively contribute to the cleaning and preprocessing of text data, making it more suitable for subsequent analysis tasks such as text summarization, sentiment analysis, and topic modeling.



# Define a function to clean text
Currently defining a function to clean text. The function performs several cleaning operations such as removing text starting with "DESCRIPTION OF", text related to "FIG", numbers from phrases, separating numbers and words, removing extra whitespaces, all numbers, consecutive periods, stop words, and lemmatizing tokens.

# Apply the function to the 'PATENT_ABSTRACT' and 'DESCRIPTION_TEXT' columns
Currently applying the function to the 'PATENT_ABSTRACT' and 'DESCRIPTION_TEXT' columns.

In [ ]:
def clean_text(text):
    # Remove text starting with "DESCRIPTION OF"
    text = re.sub(r'\bDESCRIPTION\s*OF\b.*?(?=\b(?:DESCRIPTION\s*OF\b|\bFIG\b))', '', text, flags=re.DOTALL)

    # Remove text related to "FIG"
    text = re.sub(r'\bFIGS?\..*?\b', '', text)

    # Remove numbers from phrases like "a gas turbine1" or "a generator9"
    text = re.sub(r'(\b(?:a|an|the)\s*[a-zA-Z]+)\d+(\b)', r'\1\2', text)

    # Separate numbers and words that are joined
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)

    # Remove all numbers
    text = re.sub(r'\d+', '', text)

    # Keep only one period if there are consecutive periods
    text = re.sub(r'\.{2,}', '.', text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word.lower() not in stop_words)

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in text.split()]

    # Join the tokens back into text
    cleaned_text = ' '.join(tokens)

    return cleaned_text.strip()

# Apply the function to the 'PATENT_ABSTRACT' and 'DESCRIPTION_TEXT' columns
extracted_data_summ['Concatenated_Text'] = extracted_data_summ['Concatenated_Text'].apply(clean_text)
extracted_data_train['Concatenated_Text'] = extracted_data_train['Concatenated_Text'].apply(clean_text)

In [ ]:
# Print the cleaned DataFrame
extracted_data_train

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555473,dual bladder fuel tank,2023-01-17,"patent_title: dual bladder fuel tank, patent_a..."
1,11555474,pipe section of a common rail line and methods...,2023-01-17,patent_title: pipe section common rail line me...
2,11555475,turbine,2023-01-17,"patent_title: turbine, patent_abstract: turbin..."
3,11555476,wind turbine blade root attachment system and ...,2023-01-17,patent_title: wind turbine blade root attachme...
4,11555477,bird or bat detection and identification for w...,2023-01-17,patent_title: bird bat detection identificatio...
5,11555478,deep-sea multi-energy integrated platform for ...,2023-01-17,patent_title: deep-sea multi-energy integrated...
6,11555479,portable wind energy conversion system and rel...,2023-01-17,patent_title: portable wind energy conversion ...
7,11555480,prestressed-bolted dry-assembled segmental pre...,2023-01-17,patent_title: prestressed-bolted dry-assembled...
8,11555481,inspection device controller for an inspection...,2023-01-17,patent_title: inspection device controller ins...
9,11555482,stepped conductivity interface,2023-01-17,"patent_title: stepped conductivity interface, ..."


In [ ]:
# Print the cleaned DataFrame
extracted_data_summ

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Concatenated_Text
0,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,patent_title: oldham coupling co-rotating scro...
1,11555495,fluid equalisation for multiple compressors,2023-01-17,patent_title: fluid equalisation multiple comp...
2,11555496,centrifugal pump,2023-01-17,"patent_title: centrifugal pump, patent_abstrac..."
3,11555497,quick startup device for centrifugal pump,2023-01-17,patent_title: quick startup device centrifugal...
4,11555498,"magnetic coupling assemblies and pump, turbine...",2023-01-17,"patent_title: magnetic coupling assembly pump,..."
5,11555499,centrifugal pump assembly,2023-01-17,"patent_title: centrifugal pump assembly, paten..."
6,11555500,guide vane,2023-01-17,"patent_title: guide vane, patent_abstract: pre..."
7,11555501,ceiling fan structure,2023-01-17,"patent_title: ceiling fan structure, patent_ab..."
8,11555502,compressor including inlet guide vanes,2023-01-17,patent_title: compressor including inlet guide...
9,11555503,axial counterbalance for rotating components,2023-01-17,patent_title: axial counterbalance rotating co...


**Chunking in Our Architecture**

Chunking enhances efficiency and accuracy in patent text analysis by breaking down large text into smaller segments while maintaining semantic integrity.

**Evaluation of Chunking Approaches:**

- **Fixed-Size Chunking:** Disrupts semantic coherence, lacks scalability.
- **Dynamic Chunking:** Complex implementation, inconsistent chunk sizes.
- **Sliding Window Chunking:** Chosen for enhanced semantic integrity, flexibility, and efficiency.

**Pros:**

- *Enhanced Efficiency:* Allows for parallel processing, speeding up analysis.
- *Semantic Integrity:* Maintains context across segments.
- *Scalability:* Handles growing datasets without performance loss.
- *Flexible Parameterization:* Adjusts chunk size and overlap as needed.

**Cons:**

- *Potential Data Redundancy:* Overlapping chunks may lead to redundancy.
- *Complexity in Implementation:* Adds complexity to the system.
- *Effectiveness Varies:* Varies based on dataset characteristics.

**Trade-Off:**

Introduction of redundancy is a necessary compromise for accurate analysis.

**Conclusion:**

Sliding window chunking enhances efficiency and effectiveness in patent analysis, maintaining semantic integrity for accurate recommendations.




# Applying chunking on the dataset
- Define the chunking parameters:
  - Define `chunk_size` as approximately 2000 characters.
  - Set `overlap` to 25 words between adjacent chunks.

- Create a function `chunk_text` to chunk the text by words:
  - Split the text into words.
  - Generate chunks of text based on the specified chunk size and overlap.

- Apply chunking to `Concatenated_Text` column in `extracted_data_summ`:
  - Iterate over each row in `extracted_data_summ`.
  - Chunk the text and store the chunks along with relevant metadata in `chunked_data_summ`.

- Apply chunking to `Concatenated_Text` column in `extracted_data_train`:
  - Iterate over each row in `extracted_data_train`.
  - Chunk the text and store the chunks along with relevant metadata in `chunked_data_train`.

- Convert lists of chunked data to dataframes:
  - Convert `chunked_data_summ` to `chunked_df_summ`.
  - Convert `chunked_data_train` to `chunked_df_train`.

- Print the total number of chunks generated for each row:
  - Display the total number of chunks generated for each row in `extracted_data_summ` and `extracted_data_train`.


In [ ]:
# Define the chunking parameters
chunk_size = 2000  # Approximate size of each chunk in characters
overlap = 25  # Number of overlapping words between adjacent chunks

# Function to chunk the text by words
def chunk_text(text, chunk_size, overlap):
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = min(start + chunk_size, len(words))
        chunk = ' '.join(words[start:end])
        chunks.append(chunk)
        start = start + chunk_size - overlap

    return chunks

# Create empty lists to store chunked data
chunked_data_summ = []
chunked_data_train = []

# Apply chunking to concatenated_df_summ
print("Chunking for extracted_data_summ started.")
for index, row in extracted_data_summ.iterrows():
    print(f"Chunking for row {index+1} started.")
    chunks = chunk_text(row['Concatenated_Text'], chunk_size, overlap)
    for i, chunk in enumerate(chunks):
        chunked_data_summ.append({
            'Chunk_ID': f'Chunk_{index+1}_{i+1}',
            'PATENT_ID': row['PATENT_ID'],
            'PATENT_TITLE': row['PATENT_TITLE'],
            'PATENT_DATE': row['PATENT_DATE'],
            'Text': chunk,
            'Original_Row': index+1
        })
    print(f"Chunking for row {index+1} done. Total chunks: {len(chunks)}, Chunk size: {chunk_size}")
print("Chunking for concatenated_df_summ completed.")

# Apply chunking to concatenated_df_train
print("\nChunking for extracted_data_train started.")
for index, row in extracted_data_train.iterrows():
    print(f"Chunking for row {index+1} started.")
    chunks = chunk_text(row['Concatenated_Text'], chunk_size, overlap)
    for i, chunk in enumerate(chunks):
        chunked_data_train.append({
            'Chunk_ID': f'Chunk_{index+1}_{i+1}',
            'PATENT_ID': row['PATENT_ID'],
            'PATENT_TITLE': row['PATENT_TITLE'],
            'PATENT_DATE': row['PATENT_DATE'],
            'Text': chunk,
            'Original_Row': index+1
        })
    print(f"Chunking for row {index+1} done. Total chunks: {len(chunks)}, Chunk size: {chunk_size}")
print("Chunking for extracted_data_train completed.")

# Convert lists to dataframes
chunked_df_summ = pd.DataFrame(chunked_data_summ)
chunked_df_train = pd.DataFrame(chunked_data_train)

# Print total number of chunks for each row
print("\nTotal number of chunks for extracted_data_summ:")
print(chunked_df_summ.groupby('Original_Row').size().tolist())
print("\nTotal number of chunks for extracted_data_train:")
print(chunked_df_train.groupby('Original_Row').size().tolist())

Chunking for extracted_data_summ started.
Chunking for row 1 started.
Chunking for row 1 done. Total chunks: 3, Chunk size: 2000
Chunking for row 2 started.
Chunking for row 2 done. Total chunks: 1, Chunk size: 2000
Chunking for row 3 started.
Chunking for row 3 done. Total chunks: 1, Chunk size: 2000
Chunking for row 4 started.
Chunking for row 4 done. Total chunks: 1, Chunk size: 2000
Chunking for row 5 started.
Chunking for row 5 done. Total chunks: 1, Chunk size: 2000
Chunking for row 6 started.
Chunking for row 6 done. Total chunks: 1, Chunk size: 2000
Chunking for row 7 started.
Chunking for row 7 done. Total chunks: 1, Chunk size: 2000
Chunking for row 8 started.
Chunking for row 8 done. Total chunks: 1, Chunk size: 2000
Chunking for row 9 started.
Chunking for row 9 done. Total chunks: 1, Chunk size: 2000
Chunking for row 10 started.
Chunking for row 10 done. Total chunks: 1, Chunk size: 2000
Chunking for concatenated_df_summ completed.

Chunking for extracted_data_train starte

In [ ]:
chunked_df_train

,Chunk_ID,PATENT_ID,PATENT_TITLE,PATENT_DATE,Text,Original_Row
0,Chunk_1_1,11555473,dual bladder fuel tank,2023-01-17,"patent_title: dual bladder fuel tank, patent_a...",1
1,Chunk_2_1,11555474,pipe section of a common rail line and methods...,2023-01-17,patent_title: pipe section common rail line me...,2
2,Chunk_3_1,11555475,turbine,2023-01-17,"patent_title: turbine, patent_abstract: turbin...",3
3,Chunk_4_1,11555476,wind turbine blade root attachment system and ...,2023-01-17,patent_title: wind turbine blade root attachme...,4
4,Chunk_5_1,11555477,bird or bat detection and identification for w...,2023-01-17,patent_title: bird bat detection identificatio...,5
5,Chunk_5_2,11555477,bird or bat detection and identification for w...,2023-01-17,"behavior, type behavior, combination thereof. ...",5
6,Chunk_6_1,11555478,deep-sea multi-energy integrated platform for ...,2023-01-17,patent_title: deep-sea multi-energy integrated...,6
7,Chunk_7_1,11555479,portable wind energy conversion system and rel...,2023-01-17,patent_title: portable wind energy conversion ...,7
8,Chunk_7_2,11555479,portable wind energy conversion system and rel...,2023-01-17,given frame slot may configured given conversi...,7
9,Chunk_8_1,11555480,prestressed-bolted dry-assembled segmental pre...,2023-01-17,patent_title: prestressed-bolted dry-assembled...,8


In [ ]:
chunked_df_summ

,Chunk_ID,PATENT_ID,PATENT_TITLE,PATENT_DATE,Text,Original_Row
0,Chunk_1_1,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,patent_title: oldham coupling co-rotating scro...,1
1,Chunk_1_2,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,"slot. additionally, adequate clearance inner f...",1
2,Chunk_1_3,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,"center mass maintained plane f, across driver ...",1
3,Chunk_2_1,11555495,fluid equalisation for multiple compressors,2023-01-17,patent_title: fluid equalisation multiple comp...,2
4,Chunk_3_1,11555496,centrifugal pump,2023-01-17,"patent_title: centrifugal pump, patent_abstrac...",3
5,Chunk_4_1,11555497,quick startup device for centrifugal pump,2023-01-17,patent_title: quick startup device centrifugal...,4
6,Chunk_5_1,11555498,"magnetic coupling assemblies and pump, turbine...",2023-01-17,"patent_title: magnetic coupling assembly pump,...",5
7,Chunk_6_1,11555499,centrifugal pump assembly,2023-01-17,"patent_title: centrifugal pump assembly, paten...",6
8,Chunk_7_1,11555500,guide vane,2023-01-17,"patent_title: guide vane, patent_abstract: pre...",7
9,Chunk_8_1,11555501,ceiling fan structure,2023-01-17,"patent_title: ceiling fan structure, patent_ab...",8


In patent analysis, Retrieval-Augmented Generation (RAG) combines retrieval-based methods with generative models to produce concise summaries. After evaluating various RAG models, Google Pegasus was chosen for its superior performance.

### RAG Models Tried Before Finalizing

**RAG Models Used:**

- Blaise-g/longt5_tglobal_large_scitldr
- 0x70DA/t5-v1_1-base-finetuned-sci_summ
- sambydlo/bart-large-scientific-lay-summarisation
- sambydlo/scientific_abstract_simplification-scientific-lay-summarise
- pszemraj/long-t5-tglobal-base-sci-simplify
- pszemraj/long-t5-tglobal-base-sci-simplify-elife
- chizhikchi/sci-five-radsum23
- pszemraj/long-t5-tglobal-xl-sci-simplify-elife
- AhiyaB/mt5-small-finetuned-Big-Patent-h
- anferico/bert-for-patents

**Pros of RAG:**

- **Integration of Retrieval and Generation:** RAG seamlessly integrates retrieval and generation mechanisms, ensuring relevance and accuracy.
- **Flexibility and Customization:** RAG models can be fine-tuned for specific domain needs, optimizing performance.
- **Semantic Understanding:** RAG, like Google Pegasus, captures semantic meaning for nuanced summarization.

**Cons of RAG:**

- *Computational Complexity:* Training and fine-tuning RAG models can be computationally intensive.
- *Data Dependency:* The performance of RAG models is highly dependent on the quality and diversity of the training data.
- *Interpretability:* The inner workings of RAG models can be complex and difficult to interpret.

### Conclusion

Despite its challenges, Google Pegasus emerged as the optimal choice for our RAG model, providing a powerful combination of performance, adaptability, and domain relevance.


##Fine-tuning PEGASUS Model for Text Summarization
- Load pretrained PEGASUS model and tokenizer:
  - Load the PEGASUS model named "google/pegasus-large" and its corresponding tokenizer.

- Ensure both dataframes have the same length:
  - Determine the minimum length between `chunked_df_train` and `chunked_df_summ`.
  - Truncate both dataframes to match the minimum length.

- Split the data into training and validation sets:
  - Define a split ratio of 0.8 for the training set.
  - Split the data into training and evaluation sets based on the split index.

- Tokenize the training and evaluation data:
  - Tokenize the training text and summaries using the tokenizer.
  - Perform truncation and padding with maximum lengths of 256 and 128 characters for input and output sequences, respectively.

- Create datasets:
  - Convert tokenized inputs and labels into datasets using the `Dataset.from_dict` method.

- Define training arguments:
  - Set various training arguments such as output directory, evaluation strategy, learning rate, batch sizes, number of epochs, weight decay, etc.
  - Enable mixed precision training and gradient accumulation to simulate a larger batch size.

- Define the Trainer object:
  - Instantiate a Trainer object with the specified model, training arguments, and datasets.

- Fine-tune the model:
  - Train the model using the `train` method of the Trainer object.

- Save the fine-tuned model and tokenizer:
  - Save the fine-tuned model and tokenizer to the specified directories.

In [ ]:
# Load pretrained PEGASUS model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Ensure both dataframes have the same length
min_length = min(len(chunked_df_train), len(chunked_df_summ))
chunked_df_train = chunked_df_train[:min_length]
chunked_df_summ = chunked_df_summ[:min_length]

# Split the data into training and validation sets
split_ratio = 0.8
split_index = int(min_length * split_ratio)

train_texts = chunked_df_train['Text'].tolist()[:split_index]
train_summaries = chunked_df_summ['Text'].tolist()[:split_index]

eval_texts = chunked_df_train['Text'].tolist()[split_index:]
eval_summaries = chunked_df_summ['Text'].tolist()[split_index:]

# Tokenize the training data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
train_labels = tokenizer(train_summaries, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Tokenize the evaluation data
eval_encodings = tokenizer(eval_texts, truncation=True, padding=True, max_length=256, return_tensors="pt")
eval_labels = tokenizer(eval_summaries, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Create datasets
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "labels": train_labels["input_ids"]})
eval_dataset = Dataset.from_dict({"input_ids": eval_encodings["input_ids"], "attention_mask": eval_encodings["attention_mask"], "labels": eval_labels["input_ids"]})

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
)

# Define the Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("./fine_tuned_pegasus_model")
tokenizer.save_pretrained("./fine_tuned_pegasus_tokenizer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,4.946852
1,No log,4.934246
2,No log,4.922582


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 256, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


('./fine_tuned_pegasus_tokenizer/tokenizer_config.json',
 './fine_tuned_pegasus_tokenizer/special_tokens_map.json',
 './fine_tuned_pegasus_tokenizer/spiece.model',
 './fine_tuned_pegasus_tokenizer/added_tokens.json')

### Loading and Preprocessing Data

- Load pretrained PEGASUS model and tokenizer:
  - Load the PEGASUS model named "google/pegasus-large" and its corresponding tokenizer.

- Ensure both dataframes have the same length:
  - Determine the minimum length between `chunked_df_train` and `chunked_df_summ`.
  - Truncate both dataframes to match the minimum length.

- Split the data into training and validation sets:
  - Define a split ratio of 0.8 for the training set.
  - Split the data into training and evaluation sets based on the split index.

### Tokenization

- Tokenize the training and evaluation data:
  - Tokenize the training text and summaries using the tokenizer.
  - Perform truncation and padding with maximum lengths of 256 and 128 characters for input and output sequences, respectively.

### Dataset Creation

- Create datasets:
  - Convert tokenized inputs and labels into datasets using the `Dataset.from_dict` method.

### Model Training

- Define training arguments:
  - Set various training arguments such as output directory, evaluation strategy, learning rate, batch sizes, number of epochs, weight decay, etc.
  - Enable mixed precision training and gradient accumulation to simulate a larger batch size.

- Define the Trainer object:
  - Instantiate a Trainer object with the specified model, training arguments, and datasets.

- Fine-tune the model:
  - Train the model using the `train` method of the Trainer object.

### Model Saving

- Save the fine-tuned model and tokenizer:
  - Save the fine-tuned model and tokenizer to the specified directories.


In [ ]:
# Load pretrained PEGASUS model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Summarize each chunk individually
summaries = []
patent_ids = []
patent_dates = []
patent_titles = []  # List to store patent titles

for i, row in chunked_df_summ.iterrows():
    patent_id = row['PATENT_ID']
    patent_date = row['PATENT_DATE']
    patent_title = row['PATENT_TITLE']  # Get the patent title for each row
    text = row['Text']
    print(f"Summarizing PATENT_ID {patent_id}, Chunk_ID {row['Chunk_ID']}...")
    # Tokenize the input text and move to the device (GPU if available)
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    # Generate the summary
    output_ids = model.generate(inputs["input_ids"])
    # Decode the generated summary
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    patent_ids.append(patent_id)
    patent_dates.append(patent_date)
    patent_titles.append(patent_title)  # Append the patent title for each row
    print(f"Summarizing PATENT_ID {patent_id}, Chunk_ID {row['Chunk_ID']} complete.")

# Create a DataFrame to store the summarized text
summ_df = pd.DataFrame({
    'PATENT_ID': patent_ids,
    'PATENT_TITLE': patent_titles,  # Assign the patent titles to the DataFrame
    'PATENT_DATE': patent_dates,
    'Summary': summaries
})

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_1...
Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_1 complete.
Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_2...
Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_2 complete.
Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_3...
Summarizing PATENT_ID 11555494, Chunk_ID Chunk_1_3 complete.
Summarizing PATENT_ID 11555495, Chunk_ID Chunk_2_1...
Summarizing PATENT_ID 11555495, Chunk_ID Chunk_2_1 complete.
Summarizing PATENT_ID 11555496, Chunk_ID Chunk_3_1...
Summarizing PATENT_ID 11555496, Chunk_ID Chunk_3_1 complete.
Summarizing PATENT_ID 11555497, Chunk_ID Chunk_4_1...
Summarizing PATENT_ID 11555497, Chunk_ID Chunk_4_1 complete.
Summarizing PATENT_ID 11555498, Chunk_ID Chunk_5_1...
Summarizing PATENT_ID 11555498, Chunk_ID Chunk_5_1 complete.
Summarizing PATENT_ID 11555499, Chunk_ID Chunk_6_1...
Summarizing PATENT_ID 11555499, Chunk_ID Chunk_6_1 complete.
Summarizing PATENT_ID 11555500, Chunk_ID Chunk_7_1...
Summarizing PATENT_ID 1155

In [ ]:
summ_df

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Summary
0,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,patent_title: oldham coupling co-rotating scro...
1,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,"implementations, oldham coupling includes driv..."
2,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,following symbol defined: fd=operating force f...
3,11555495,fluid equalisation for multiple compressors,2023-01-17,patent_title: fluid equalisation multiple comp...
4,11555496,centrifugal pump,2023-01-17,"patent_title: centrifugal pump, patent_abstrac..."
5,11555497,quick startup device for centrifugal pump,2023-01-17,spring separation baffle arranged accommodatio...
6,11555498,"magnetic coupling assemblies and pump, turbine...",2023-01-17,"patent_title: magnetic coupling assembly pump,..."
7,11555499,centrifugal pump assembly,2023-01-17,least part valve element () additionally movab...
8,11555500,guide vane,2023-01-17,"patent_title: guide vane, patent_abstract: pre..."
9,11555501,ceiling fan structure,2023-01-17,middle portion top hanging plate recessed down...


### Evaluation Metrics Calculation

- Import necessary libraries:
  - Import functions for calculating ROUGE scores, BLEU score, and BERTScore.

- Define functions for evaluation metrics calculation:
  - Define a function to calculate ROUGE scores using the RougeScorer from `rouge_score`.
  - Define a function to calculate BLEU score using the `corpus_bleu` function from `nltk.translate.bleu_score`.
  - Define a function to calculate BERTScore using the `score` function from `bert_score`.

- Extract hypotheses and references:
  - Extract generated summaries (`hypotheses`) and original document content (`references`) from the data.

- Calculate evaluation scores:
  - Iterate through each pair of hypothesis and reference.
  - Calculate ROUGE scores, BLEU score, and BERTScore for each pair.
  - Store all scores in separate lists.

- Print evaluation scores:
  - Print the calculated ROUGE scores, BLEU scores, and BERTScores.

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score as bert_score_fn

# Function to calculate ROUGE scores
def calculate_rouge_scores(hypothesis, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(hypothesis, reference)
    return rouge_scores

# Function to calculate BLEU score
def calculate_bleu_score(hypothesis, reference):
    return corpus_bleu([[reference.split()]], [hypothesis.split()])

# Function to calculate BERTScore
def calculate_bert_score(hypothesis, reference):
    P, R, F1 = bert_score_fn([hypothesis], [reference], lang="en")  # Use renamed function
    return {"Precision": P.item(), "Recall": R.item(), "F1 Score": F1.item()}

# Extract hypotheses (generated summaries) and references (original document content)
hypotheses = summ_df['Summary'].tolist()
references = chunked_df_summ['Text'].tolist()

# Calculate ROUGE scores, BLEU score, and BERTScore for each pair of hypothesis and reference
all_rouge_scores = []
all_bleu_scores = []
all_bert_scores = []

for hypothesis, reference in zip(hypotheses, references):
    rouge_scores = calculate_rouge_scores(hypothesis, reference)
    bleu_score = calculate_bleu_score(hypothesis, reference)
    bert_score = calculate_bert_score(hypothesis, reference)

    all_rouge_scores.append(rouge_scores)
    all_bleu_scores.append(bleu_score)
    all_bert_scores.append(bert_score)

# Print scores
print("ROUGE Scores:", all_rouge_scores)
print("BLEU Scores:", all_bleu_scores)
print("BERTScores:", all_bert_scores)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

ROUGE Scores: [{'rouge1': Score(precision=0.04583538689009364, recall=1.0, fmeasure=0.0876531573986805), 'rouge2': Score(precision=0.04487179487179487, recall=0.9891304347826086, fmeasure=0.08584905660377358), 'rougeL': Score(precision=0.04583538689009364, recall=1.0, fmeasure=0.0876531573986805)}, {'rouge1': Score(precision=0.04811507936507937, recall=1.0, fmeasure=0.09181258873639377), 'rouge2': Score(precision=0.04665012406947891, recall=0.9791666666666666, fmeasure=0.08905731880625296), 'rougeL': Score(precision=0.04811507936507937, recall=1.0, fmeasure=0.09181258873639377)}, {'rouge1': Score(precision=0.19038461538461537, recall=1.0, fmeasure=0.3198707592891761), 'rouge2': Score(precision=0.18497109826589594, recall=0.9795918367346939, fmeasure=0.31118314424635335), 'rougeL': Score(precision=0.19038461538461537, recall=1.0, fmeasure=0.3198707592891761)}, {'rouge1': Score(precision=0.20072115384615385, recall=1.0, fmeasure=0.3343343343343344), 'rouge2': Score(precision=0.1985559566

### Evaluation Metrics

Evaluation metrics provide insights into the performance of text generation or summarization systems:

- **ROUGE Score:** Measures overlap between generated and reference text, focusing on recall.
- **BLEU Score:** Measures n-gram overlap between candidate and reference text.
- **BERT Score:** Measures semantic similarity using BERT embeddings.
- **Cross-Validation:** Ensures robustness and generalizability of model recommendations.

#### ROUGE Scores

- High recall (1.0) indicates comprehensive coverage but low precision suggests verbosity.
- F-measure balances precision and recall.

#### BLEU Scores

- Near-zero scores indicate significant divergence from the reference.
- Higher scores suggest better n-gram matches.

#### BERT Scores

- High precision, recall, and F1 scores indicate semantic similarity despite exact wording differences.
- Consistency in scores reflects the model's ability to capture semantic essence.

### Analysis

- **ROUGE:** Perfect recall with low precision suggests verbosity. Improve precision while maintaining high recall.
- **BLEU:** Low scores indicate divergence in wording and structure. Improve fluency and coherence.
- **BERT Scores:** High scores indicate semantic similarity. Aim for a better balance between content coverage and textual precision.

### Recommendations for Improvement

- Reduce verbosity to improve precision.
- Enhance fluency and structure for better readability.
- Strive for a balance between content coverage and textual precision.

By addressing these areas, overall system performance can be enhanced.


### Model Initialization and Preprocessing

- Suppress initialization warning:
  - Suppress the warning about model initialization to reduce verbosity.

- Load pretrained PEGASUS model and tokenizer:
  - Load the PEGASUS model named "google/pegasus-large" and its corresponding tokenizer.

### Perplexity Score Calculation

- Define a function to calculate perplexity scores:
  - Define a function to calculate perplexity scores for each summary.
  - Tokenize the text using the tokenizer.
  - Generate logits using the PEGASUS model.
  - Calculate perplexity based on the cross-entropy loss.

- Calculate perplexity scores:
  - Iterate through each summary.
  - Calculate perplexity scores using the defined function.
  - Print the calculated perplexity scores.

In [ ]:
# Suppress the warning about model initialization
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load pretrained PEGASUS model and tokenizer
model_name = "google/pegasus-large"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

# Function to calculate perplexity score for each summary
def calculate_perplexity_scores(summaries):
    perplexity_scores = []
    for summary in summaries:
        # Tokenize the text
        input_ids = tokenizer(summary, return_tensors="pt", max_length=512, truncation=True)["input_ids"]
        # Generate logits
        with torch.no_grad():
            # Add decoder_input_ids explicitly
            logits = model(input_ids, decoder_input_ids=input_ids).logits
        # Calculate perplexity
        perplexity = torch.exp(torch.nn.functional.cross_entropy(logits.view(-1, logits.size(-1)), input_ids.view(-1)))
        perplexity_scores.append(perplexity.item())
    return perplexity_scores

# Calculate perplexity scores for each summary
perplexity_scores = calculate_perplexity_scores(summ_df['Summary'])
print("Perplexity Scores:", perplexity_scores)


Perplexity Scores: [21161.888671875, 26908.2109375, 4310.64208984375, 7491.19775390625, 89676.9296875, 672.90478515625, 13338.0849609375, 1073.058349609375, 11824.3623046875, 2805.924560546875, 17811.07421875, 1459.02392578125]


In [ ]:
summ_df

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Summary
0,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,patent_title: oldham coupling co-rotating scro...
1,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,"implementations, oldham coupling includes driv..."
2,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,following symbol defined: fd=operating force f...
3,11555495,fluid equalisation for multiple compressors,2023-01-17,patent_title: fluid equalisation multiple comp...
4,11555496,centrifugal pump,2023-01-17,"patent_title: centrifugal pump, patent_abstrac..."
5,11555497,quick startup device for centrifugal pump,2023-01-17,spring separation baffle arranged accommodatio...
6,11555498,"magnetic coupling assemblies and pump, turbine...",2023-01-17,"patent_title: magnetic coupling assembly pump,..."
7,11555499,centrifugal pump assembly,2023-01-17,least part valve element () additionally movab...
8,11555500,guide vane,2023-01-17,"patent_title: guide vane, patent_abstract: pre..."
9,11555501,ceiling fan structure,2023-01-17,middle portion top hanging plate recessed down...


### Displaying Data

- Import necessary libraries:
  - Import the `display` function from `IPython.display`.

- Display the entire PATENT_TITLE column without truncation:
  - Use the `display` function to show the entire content of the `PATENT_TITLE` column in `summ_df` without truncation.

In [ ]:
from IPython.display import display # Display the entire PATENT_ID column without truncation
display(summ_df['PATENT_TITLE'])

0     oldham coupling in co-rotating scroll compressors
1     oldham coupling in co-rotating scroll compressors
2     oldham coupling in co-rotating scroll compressors
3           fluid equalisation for multiple compressors
4                                      centrifugal pump
5             quick startup device for centrifugal pump
6     magnetic coupling assemblies and pump, turbine...
7                             centrifugal pump assembly
8                                            guide vane
9                                 ceiling fan structure
10               compressor including inlet guide vanes
11         axial counterbalance for rotating components
Name: PATENT_TITLE, dtype: object

### Data Grouping and Concatenation

- Group by PATENT_ID and concatenate summaries:
  - Group the data in `summ_df` by the `PATENT_ID` column.
  - Aggregate the grouped data:
    - Select the first occurrence of `PATENT_TITLE` and `PATENT_DATE`.
    - Concatenate all summaries with a space.
  - Reset the index of the resulting DataFrame.

- Display the result:
  - Print the grouped DataFrame (`grouped_df`) showing the aggregated data.

In [ ]:
# Group by PATENT_ID and concatenate summaries
grouped_df = summ_df.groupby('PATENT_ID').agg({
    'PATENT_TITLE': 'first',
    'PATENT_DATE': 'first',
    'Summary': ' '.join  # Concatenate summaries with a space
}).reset_index()

# Display the result
print(grouped_df)

   PATENT_ID                                       PATENT_TITLE PATENT_DATE  \
0   11555494  oldham coupling in co-rotating scroll compressors  2023-01-17   
1   11555495        fluid equalisation for multiple compressors  2023-01-17   
2   11555496                                   centrifugal pump  2023-01-17   
3   11555497          quick startup device for centrifugal pump  2023-01-17   
4   11555498  magnetic coupling assemblies and pump, turbine...  2023-01-17   
5   11555499                          centrifugal pump assembly  2023-01-17   
6   11555500                                         guide vane  2023-01-17   
7   11555501                              ceiling fan structure  2023-01-17   
8   11555502             compressor including inlet guide vanes  2023-01-17   
9   11555503       axial counterbalance for rotating components  2023-01-17   

                                             Summary  
0  patent_title: oldham coupling co-rotating scro...  
1  patent_title: flu

In [ ]:
grouped_df

,PATENT_ID,PATENT_TITLE,PATENT_DATE,Summary
0,11555494,oldham coupling in co-rotating scroll compressors,2023-01-17,patent_title: oldham coupling co-rotating scro...
1,11555495,fluid equalisation for multiple compressors,2023-01-17,patent_title: fluid equalisation multiple comp...
2,11555496,centrifugal pump,2023-01-17,"patent_title: centrifugal pump, patent_abstrac..."
3,11555497,quick startup device for centrifugal pump,2023-01-17,spring separation baffle arranged accommodatio...
4,11555498,"magnetic coupling assemblies and pump, turbine...",2023-01-17,"patent_title: magnetic coupling assembly pump,..."
5,11555499,centrifugal pump assembly,2023-01-17,least part valve element () additionally movab...
6,11555500,guide vane,2023-01-17,"patent_title: guide vane, patent_abstract: pre..."
7,11555501,ceiling fan structure,2023-01-17,middle portion top hanging plate recessed down...
8,11555502,compressor including inlet guide vanes,2023-01-17,patent_title: compressor including inlet guide...
9,11555503,axial counterbalance for rotating components,2023-01-17,axial counterbalance allow larger range axial ...


### Why Pegasus was Considered Over BERT: Embeddings generated through Pegasus

#### Architecture Differences

**BERT:**
- Designed for understanding tasks, not text generation.
- Generates context-aware embeddings but not new sequences like summaries.

**Pegasus:**
- Specifically designed for text generation tasks like summarization.
- Uses a pre-training objective tailored for summarization, enabling new sequence generation.

#### Pre-training Objectives

**BERT:**
- Uses Masked Language Model (MLM) and Next Sentence Prediction (NSP) objectives.
- Predicts masked tokens and sentence relationships.

**Pegasus:**
- Uses Gap Sentences Generation (GSG) pre-training objective.
- Trained to generate masked sentences directly, highly effective for summarization.

#### Fine-tuning for Summarization

**BERT:**
- Can be fine-tuned but requires additional mechanisms for generation tasks.

**Pegasus:**
- Directly supports fine-tuning for text generation tasks, making it more efficient.

#### Embedding Use Cases

**BERT:**
- Great for understanding context within sentences, used for classification and labeling.

**Pegasus:**
- Optimized for generating coherent and contextually appropriate new text sequences.

### Conclusion

Pegasus's architecture and pre-training objectives provide a significant advantage for generating new text sequences like summaries, making it superior to BERT.

## Loading Fine-tuned PEGASUS Model and Tokenizer

- Load fine-tuned PEGASUS model and tokenizer:
  - Specify the paths for the fine-tuned model (`model_path`) and tokenizer (`tokenizer_path`).
  - Load the tokenizer using the specified path.
  - Load the model using the specified path.

- Move the model to GPU if available:
  - Check if a GPU is available.
  - Move the model to the GPU if one is available; otherwise, use the CPU.

In [ ]:
# Load fine-tuned PEGASUS model and tokenizer
model_path = "./fine_tuned_pegasus_model"
tokenizer_path = "./fine_tuned_pegasus_tokenizer"
tokenizer = PegasusTokenizer.from_pretrained(tokenizer_path)
model = PegasusForConditionalGeneration.from_pretrained(model_path)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

## Function for Generating Embeddings

- Define a function to generate embeddings for a list of summaries:
  - Define a function named `generate_embeddings` that takes a list of summaries as input.
  - Iterate through each summary in the input list.
  - Tokenize the summary using the tokenizer and move the tensors to the specified device (GPU/CPU).
  - Generate embeddings using the PEGASUS model's encoder.
  - Calculate the mean of encoder hidden states to obtain the embedding.
  - Convert the embedding to a NumPy array and append it to the list of embeddings.
  - If no encoder hidden states are found for a summary, print a warning message.

- This function returns a list of embeddings for the input summaries.


In [ ]:
# Define a function to generate embeddings for a list of summaries
def generate_embeddings(summaries):
    embeddings = []
    for summary in summaries:
        inputs = tokenizer(summary, return_tensors="pt", max_length=256, truncation=True, padding=True).to(device)
        with torch.no_grad():
            outputs = model.model.encoder(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        encoder_hidden_states = outputs.last_hidden_state
        if encoder_hidden_states is not None:
            embedding = encoder_hidden_states.mean(dim=1).cpu().detach().numpy()
            embeddings.append(embedding)
        else:
            print(f"Warning: No encoder hidden states for summary '{summary}'")
    return embeddings

## Generating Embeddings for Summaries

- Generate embeddings for all summaries in `grouped_df`:
  - Extract the summaries from the `Summary` column of `grouped_df` and convert them to a list.
  - Call the `generate_embeddings` function with the list of summaries as input.
  - This step generates embeddings for all the summaries in `grouped_df`.

In [ ]:
# Generate embeddings for all summaries in grouped_df
summaries = grouped_df['Summary'].tolist()
embeddings = generate_embeddings(summaries)

## Printing Debug Information

- Print debug information:
  - Print the total number of embeddings generated.
  - Iterate through each embedding and print its shape and type.
  - This step provides debug information to verify the correctness of the generated embeddings.


In [ ]:
# Print debug information
print(f"Generated {len(embeddings)} embeddings.")
for i, embedding in enumerate(embeddings):
    print(f"Embedding {i}: shape={embedding.shape}, type={type(embedding)}")

Generated 10 embeddings.
Embedding 0: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 1: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 2: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 3: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 4: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 5: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 6: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 7: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 8: shape=(1, 1024), type=<class 'numpy.ndarray'>
Embedding 9: shape=(1, 1024), type=<class 'numpy.ndarray'>


## Converting Embeddings to NumPy Array and Adding to DataFrame

- Convert list of embeddings to a NumPy array and add to DataFrame:
  - Use `np.vstack` to vertically stack the list of embeddings into a single NumPy array.
  - Assign the array to a new column named 'Embeddings' in the DataFrame `grouped_df`.
  - This step converts the embeddings into a format suitable for further analysis and adds them to the DataFrame.

- Print confirmation message:
  - Print a message to indicate that embeddings have been generated and added to the DataFrame.


In [ ]:
# Convert list of embeddings to a NumPy array and add to DataFrame
embeddings_array = np.vstack(embeddings)
grouped_df['Embeddings'] = embeddings

print("Embeddings generated and added to DataFrame.")

Embeddings generated and added to DataFrame.


In [ ]:
print(grouped_df['Embeddings'])

### Utilizing FAISS Database for Patent Summarization: Evaluation and Selection

We chose FAISS over Chroma for its efficiency, scalability, and advanced similarity search.

#### Why FAISS was Used?

- High Efficiency and Scalability: Optimized for large-scale similarity search, ensuring efficient retrieval of patent datasets.
- Advanced Similarity Search: Utilizes state-of-the-art algorithms for precise retrieval of relevant patent documents.
- Customizability and Flexibility: Provides extensive customization options to fine-tune indexing parameters for domain-specific needs.

#### Pros and Cons of FAISS

**Pros:**
- Efficient Retrieval: Crucial for managing extensive patent datasets.
- Scalability: Ensures system remains effective with dataset growth.
- Customizable Indexes: Supports various indexing methods for optimized performance.

**Cons:**
- Complexity: Requires deep understanding for implementation and maintenance.
- Memory Usage: Can be memory-intensive, demanding computational resources.

#### Selection of FAISS over Chroma

FAISS outperformed Chroma in performance, scalability, and customization options.

#### Benefits of Using FAISS

- Enhanced Retrieval Speed and Accuracy: Contributes to more precise and relevant summaries.
- Scalable Solution: Efficient even as the patent database expands.
- Optimized Resource Utilization: Balances performance with hardware constraints.


## Initializing and Adding Embeddings to FAISS Index

- Initialize FAISS index:
  - Determine the dimensionality of the embeddings array.
  - Initialize a flat index (`IndexFlatL2`) in FAISS with the same dimensionality as the embeddings.

- Add embeddings to the FAISS index:
  - Add the embeddings array to the FAISS index.
  - This step populates the FAISS index with the embeddings for efficient similarity search.

- Print confirmation message:
  - Print a message indicating the number of embeddings added to the FAISS index.


In [ ]:
# Initialize FAISS index
embedding_dim = embeddings_array.shape[1]
index = faiss.IndexFlatL2(embedding_dim)

# Add embeddings to the FAISS index
index.add(embeddings_array)

print(f"Added {index.ntotal} embeddings to the FAISS index.")

Added 10 embeddings to the FAISS index.


### Querying FAISS Index for Similar Summaries

- Define a function to query the FAISS index for similar summaries:
  - Define a function named `query_faiss_index` that takes a query summary and an optional parameter `top_k` (default value: 5) as input.
  - Generate embedding for the query summary using the `generate_embeddings` function.
  - Search the FAISS index for the nearest neighbors to the query embedding using the `index.search` method.
  - Return the indices of the top-k similar summaries.

- Example usage:
  - Provide an example query summary (`input_summary`) and specify the number of similar summaries to retrieve (`top_k`).
  - Call the `query_faiss_index` function with the input summary and top-k parameters.
  - Retrieve the top-k similar patents from `grouped_df` based on the indices returned by the FAISS index search.

- Print top-k similar patents:
  - Print the top-k similar patents, including their IDs, titles, and dates.

In [ ]:
def query_faiss_index(query_summary, top_k=5):
    # Generate embedding for the query summary
    query_embedding = generate_embeddings([query_summary])[0]

    # Search the FAISS index
    _, indices = index.search(query_embedding, top_k)

    return indices[0]

# Example usage:
input_summary = "centrifugal pump assembly"
top_k = 5
similar_indices = query_faiss_index(input_summary, top_k)

# Get the top-k similar patents
similar_patents = grouped_df.iloc[similar_indices]

print("Top-k similar patents:")
print(similar_patents[['PATENT_ID', 'PATENT_TITLE', 'PATENT_DATE']])


Top-k similar patents:
   PATENT_ID                               PATENT_TITLE PATENT_DATE
2   11555496                           centrifugal pump  2023-01-17
3   11555497  quick startup device for centrifugal pump  2023-01-17
5   11555499                  centrifugal pump assembly  2023-01-17
7   11555501                      ceiling fan structure  2023-01-17
8   11555502     compressor including inlet guide vanes  2023-01-17


### FAISS Index Initialization and Embedding Addition

- Initialize FAISS index with IndexHNSW:

- Add embeddings to the FAISS index:

- Print the number of embeddings added to the FAISS index:


In [ ]:
# Initialize FAISS index with IndexHNSW
index = faiss.IndexHNSWFlat(embedding_dim, 32)

# Add embeddings to the FAISS index
index.add(embeddings_array)

print(f"Added {index.ntotal} embeddings to the FAISS index.")

Added 10 embeddings to the FAISS index.


## Function: Recommend Summaries Based on Query Summary

- Define a function to recommend summaries based on a query summary:


In [ ]:
# Define a function to recommend summaries based on a query summary
def recommend_summaries(query_summary, top_k=5):
    # Generate embedding for the query summary
    query_embedding = generate_embeddings([query_summary])[0]

    # Search the FAISS index for similar embeddings
    _, indices = index.search(query_embedding, top_k)

    return indices[0]

### Example Usage: Recommending Summaries Based on Query Summary

- Define an input summary and the number of top-k recommendations:

- Use the `recommend_summaries` function to get the top-k recommended indices:
    

- Get the top-k recommended patents from the grouped DataFrame:
    

- Print the top-k recommended patents based on the query summary:
    

In [ ]:
# Example usage:
input_summary = "centrifugal pump assembly"
top_k = 5
recommended_indices = recommend_summaries(input_summary, top_k)

# Get the top-k recommended patents
recommended_summaries = grouped_df.iloc[recommended_indices]

print("Top-k recommended patents based on query summary:")
print(recommended_summaries[['PATENT_ID', 'PATENT_TITLE', 'PATENT_DATE', 'Summary']])


Top-k recommended patents based on query summary:
   PATENT_ID                               PATENT_TITLE PATENT_DATE  \
2   11555496                           centrifugal pump  2023-01-17   
3   11555497  quick startup device for centrifugal pump  2023-01-17   
5   11555499                  centrifugal pump assembly  2023-01-17   
7   11555501                      ceiling fan structure  2023-01-17   
8   11555502     compressor including inlet guide vanes  2023-01-17   

                                             Summary  
2  patent_title: centrifugal pump, patent_abstrac...  
3  spring separation baffle arranged accommodatio...  
5  least part valve element () additionally movab...  
7  middle portion top hanging plate recessed down...  
8  patent_title: compressor including inlet guide...  


### Function: Recommend Summaries Based on Query Summary Using Cosine Similarity

- Import necessary libraries
- Define a function to recommend summaries based on a query summary:
- Define a function to calculate cosine similarity between two embeddings:
- Example usage:
    ```python
    input_summary = "centrifugal pump assembly"
    top_k = 5
    similarity_scores_df = recommend_summaries(input_summary, grouped_df, embeddings, top_k)
    print("Similarity Scores:")
    print(similarity_scores_df)
    ```


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Define a function to recommend summaries based on a query summary
def recommend_summaries(query_summary, grouped_df, embeddings, top_k=5):
    # Generate embedding for the query summary
    query_embedding = generate_embeddings([query_summary])[0]

    # Search the FAISS index for similar embeddings
    _, indices = index.search(query_embedding, top_k)

    # Initialize a DataFrame to store similarity scores
    similarity_df = pd.DataFrame(columns=['PATENT_ID', 'PATENT_TITLE', 'PATENT_DATE', 'Similarity Score'])

    # Calculate similarity scores between input summary and recommended summaries
    for i, idx in enumerate(indices[0]):
        recommended_embedding = embeddings[idx]
        similarity_score = calculate_similarity(query_embedding, recommended_embedding)
        patent_info = grouped_df.iloc[idx]
        similarity_df.loc[i] = {
            'PATENT_ID': patent_info['PATENT_ID'],
            'PATENT_TITLE': patent_info['PATENT_TITLE'],
            'PATENT_DATE': patent_info['PATENT_DATE'],
            'Similarity Score': similarity_score
        }

    # Sort DataFrame by similarity score in descending order
    similarity_df = similarity_df.sort_values(by='Similarity Score', ascending=False)

    return similarity_df

# Define a function to calculate cosine similarity between two embeddings
def calculate_similarity(embedding1, embedding2):
    # Calculate cosine similarity between two embeddings
    similarity_score = cosine_similarity(embedding1.reshape(1, -1), embedding2.reshape(1, -1))[0][0]
    return similarity_score

# Example usage:
input_summary = "centrifugal pump assembly"
top_k = 5
similarity_scores_df = recommend_summaries(input_summary, grouped_df, embeddings, top_k)
print("Similarity Scores:")
print(similarity_scores_df)


Similarity Scores:
   PATENT_ID                               PATENT_TITLE PATENT_DATE  \
0   11555496                           centrifugal pump  2023-01-17   
2   11555499                  centrifugal pump assembly  2023-01-17   
1   11555497  quick startup device for centrifugal pump  2023-01-17   
3   11555501                      ceiling fan structure  2023-01-17   
4   11555502     compressor including inlet guide vanes  2023-01-17   

   Similarity Score  
0          0.540165  
2          0.505261  
1          0.497840  
3          0.480463  
4          0.473443  
